In [51]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/ncr_ride_bookings.csv')
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               150000 non-null  object 
 1   Time                               150000 non-null  object 
 2   Booking ID                         150000 non-null  object 
 3   Booking Status                     150000 non-null  object 
 4   Customer ID                        150000 non-null  object 
 5   Vehicle Type                       150000 non-null  object 
 6   Pickup Location                    150000 non-null  object 
 7   Drop Location                      150000 non-null  object 
 8   Avg VTAT                           139500 non-null  float64
 9   Avg CTAT                           102000 non-null  float64
 10  Cancelled Rides by Customer        10500 non-null   float64
 11  Reason for cancelling by Customer  1050

None

In [52]:
display(df.head())

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


In [53]:
total_bookings = df['Booking ID'].nunique()
print(f'Total Bookings: {total_bookings/1000:.2f}K')
success_rate = df[df['Booking Status'].isin(['Completed', 'Incomplete'])]['Booking ID'].nunique() / total_bookings * 100
print(f'Success Rate: {success_rate:.2f}%')
canceled_customer_rate = df[df['Booking Status'] == 'Cancelled by Customer']['Booking ID'].nunique() / total_bookings * 100
canceled_driver_rate = df[df['Booking Status'] == 'Cancelled by Driver']['Booking ID'].nunique() / total_bookings * 100
print(f'Canceled Rate: {canceled_customer_rate + canceled_driver_rate:.2f}%')
print(f' - By Customer: {canceled_customer_rate:.2f}%')
print(f' - By Driver: {canceled_driver_rate:.2f}%')

Total Bookings: 148.77K
Success Rate: 68.20%
Canceled Rate: 25.18%
 - By Customer: 7.06%
 - By Driver: 18.12%


In [54]:
df_success = df[df['Booking Status'].isin(['Completed', 'Incomplete'])]
success = df_success.groupby('Vehicle Type').agg({'Booking ID': 'count', 'Ride Distance': ['mean', 'sum']}).reset_index()
success['Ride Distance', 'mean'] = success['Ride Distance', 'mean'].round(2)
success['Ride Distance', 'sum'] = (success['Ride Distance', 'sum'] / 1000).round(2)
success.rename(columns={'count': 'Total Rides', 'mean': 'Avg Distance km', 'sum': 'Total Distance 1000km'}, inplace=True)
display(success)

Vehicle Type  Booking ID   Ride Distance                      
                 Total Rides Avg Distance km Total Distance 1000km
0           Auto       25415           24.62                625.62
1           Bike       15362           24.65                378.67
2        Go Mini       20364           24.61                501.20
3       Go Sedan       18318           24.61                450.79
4  Premier Sedan       12315           24.60                302.94
5        Uber XL        3045           24.40                 74.31
6          eBike        7181           24.99                179.46

In [55]:
df_incomplete = df[df['Booking Status'] == 'Incomplete']
incomlete = df_incomplete.groupby('Incomplete Rides Reason').agg({'Booking ID': 'count'}).reset_index()

In [56]:
df_customer_canceled = df[df['Booking Status'] == 'Cancelled by Customer']
df_driver_canceled = df[df['Booking Status'] == 'Cancelled by Driver']
customer_canceled = df_customer_canceled.groupby('Vehicle Type').agg({'Booking ID': 'count'}).reset_index()
driver_canceled = df_driver_canceled.groupby('Vehicle Type').agg({'Booking ID': 'count'}).reset_index()
total = df[df['Booking Status'] != 'No Driver Found'].groupby('Vehicle Type').agg({'Booking ID': 'count'}).reset_index()
canceled = pd.merge(customer_canceled, driver_canceled, on='Vehicle Type', how='left', suffixes=('_customer', '_driver'))
success_rate = pd.merge(total, canceled, on='Vehicle Type', how='left', suffixes=('_total', '_canceled'))
success_rate['Total Success Rate %'] = (success_rate['Booking ID'] - success_rate['Booking ID_customer'] - success_rate['Booking ID_driver']) / success_rate['Booking ID'] * 100    
success_rate['Total Success Rate %'] = success_rate['Total Success Rate %'].round(2)
success_rate.rename(columns={'Booking ID': 'Total Rides', 'Booking ID_customer': 'Cancelled by Customer', 'Booking ID_driver': 'Cancelled by Driver'}, inplace=True)
display(success_rate)

,Vehicle Type,Total Rides,Cancelled by Customer,Cancelled by Driver,Total Success Rate %
0,Auto,34738,2680,6643,73.16
1,Bike,21014,1575,4077,73.10
2,Go Mini,27791,2097,5330,73.28
3,Go Sedan,25181,1832,5031,72.75
4,Premier Sedan,16831,1266,3250,73.17
5,Uber XL,4134,327,762,73.66
6,eBike,9811,723,1907,73.19


In [57]:
payment = df[df['Payment Method'].notnull()]
payment_method = payment.groupby('Payment Method').agg({'Booking ID': 'count'}).reset_index()
payment_method['% of Total'] = payment_method['Booking ID']/payment_method['Booking ID'].sum()*100
payment_method['% of Total'] = payment_method['% of Total'].round(2)
payment_method.rename(columns={'Booking ID': 'Total Rides'}, inplace=True)
display(payment_method.sort_values('% of Total', ascending=False))

,Payment Method,Total Rides,% of Total
3,UPI,45909,45.01
0,Cash,25367,24.87
4,Uber Wallet,12276,12.04
1,Credit Card,10209,10.01
2,Debit Card,8239,8.08


In [58]:
reason_sustomer_canceled = df_customer_canceled.groupby('Reason for cancelling by Customer').agg({'Booking ID': 'count'}).reset_index()
reason_sustomer_canceled['% of Total'] = reason_sustomer_canceled['Booking ID']/reason_sustomer_canceled['Booking ID'].sum()*100
reason_sustomer_canceled['% of Total'] = reason_sustomer_canceled['% of Total'].round(2)
reason_sustomer_canceled.rename(columns={'Booking ID': 'Total Rides'}, inplace=True)
display(reason_sustomer_canceled.sort_values('% of Total', ascending=False))    

,Reason for cancelling by Customer,Total Rides,% of Total
4,Wrong Address,2362,22.50
1,Change of plans,2353,22.41
3,Driver is not moving towards pickup location,2335,22.24
2,Driver asked to cancel,2295,21.86
0,AC is not working,1155,11.00


In [59]:
reason_driver_canceled = df_driver_canceled.groupby('Driver Cancellation Reason').agg({'Booking ID': 'count'}).reset_index()
reason_driver_canceled['% of Total'] = reason_driver_canceled['Booking ID']/reason_driver_canceled['Booking ID'].sum() * 100
reason_driver_canceled['% of Total'] = reason_driver_canceled['% of Total'].round(2)
reason_driver_canceled.rename(columns={'Booking ID': 'Total Rides'}, inplace=True)
display(reason_driver_canceled.sort_values('% of Total', ascending=False))

,Driver Cancellation Reason,Total Rides,% of Total
0,Customer related issue,6837,25.32
3,The customer was coughing/sick,6751,25.00
2,Personal & Car related issues,6726,24.91
1,More than permitted people in there,6686,24.76


In [60]:
df_completed = df[df['Booking Status'] == 'Completed']
castomer_rating = df_completed.groupby(['Vehicle Type']).agg(
    avg_rating=('Customer Rating', 'mean'),
    total_rides=('Booking ID', 'count')
).reset_index()
display(castomer_rating.sort_values('avg_rating', ascending=False))

,Vehicle Type,avg_rating,total_rides
3,Go Sedan,4.409996,16676
5,Uber XL,4.404851,2783
2,Go Mini,4.404297,18549
6,eBike,4.403954,6551
1,Bike,4.403940,14034
4,Premier Sedan,4.403457,11252
0,Auto,4.402000,23155


In [61]:
driver_rating = df_completed.groupby('Vehicle Type').agg(
    avg_rating=('Driver Ratings','mean'),
    total_rides=('Booking ID','count')
).reset_index()
display(driver_rating.sort_values('avg_rating', ascending=False))

,Vehicle Type,avg_rating,total_rides
5,Uber XL,4.238340,2783
4,Premier Sedan,4.234865,11252
0,Auto,4.232369,23155
3,Go Sedan,4.231812,16676
1,Bike,4.230056,14034
2,Go Mini,4.227694,18549
6,eBike,4.225614,6551


In [62]:
routes = df_completed.groupby(['Pickup Location', 'Drop Location']).agg({'Booking ID': 'count', 'Ride Distance': 'mean'}).reset_index()
routes['Ride Distance'] = routes['Ride Distance'].round(2)
routes.rename(columns={'Booking ID': 'Total Rides', 'Ride Distance': 'Avg Distance km'}, inplace=True)
display(routes.sort_values('Total Rides', ascending=False).head(10))

,Pickup Location,Drop Location,Total Rides,Avg Distance km
4862,DLF City Court,Bhiwadi,13,30.76
23081,Rohini West,Sohna Road,13,29.41
18751,Noida Sector 62,Sarai Kale Khan,11,24.93
14383,Lok Kalyan Marg,Jhilmil,11,18.75
25992,Subhash Chowk,IGNOU Road,11,28.02
499,Ambience Mall,Akshardham,11,26.94
22594,Rithala,Udyog Vihar Phase 4,11,18.86
26589,Sushant Lok,Sarai Kale Khan,11,21.52
20269,Peeragarhi,Anand Vihar,10,28.55
25823,South Extension,Gwal Pahari,10,29.03


In [63]:
df_completed['Date'] = pd.to_datetime(df_completed['Date']) + pd.to_timedelta(df_completed['Time'])
df_completed['DayOfWeek'] = df_completed['Date'].dt.day_name()
day_of_week = df_completed.groupby('DayOfWeek').agg({'Booking ID': 'count'}).reset_index()
day_of_week['% of Total'] = day_of_week['Booking ID']/day_of_week['Booking ID'].sum()*100
day_of_week['% of Total'] = day_of_week['% of Total'].round(2)
day_of_week.rename(columns={'Booking ID': 'Total Rides'}, inplace=True)
display(day_of_week.sort_values('% of Total', ascending=False))

/var/folders/_6/k5gdhv996c7d1c62dptcsljm0000gn/T/ipykernel_84699/589074182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completed['Date'] = pd.to_datetime(df_completed['Date']) + pd.to_timedelta(df_completed['Time'])
/var/folders/_6/k5gdhv996c7d1c62dptcsljm0000gn/T/ipykernel_84699/589074182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completed['DayOfWeek'] = df_completed['Date'].dt.day_name()


,DayOfWeek,Total Rides,% of Total
2,Saturday,13442,14.45
1,Monday,13318,14.32
3,Sunday,13314,14.32
6,Wednesday,13296,14.30
0,Friday,13253,14.25
4,Thursday,13206,14.20
5,Tuesday,13171,14.16


In [64]:
bins = [0, 6 ,12, 18, 24]
labels = ['Night', 'Morning', 'Afternoon', 'Evening']
df_completed['Hour'] = df_completed['Date'].dt.hour
df_completed['Time of Day'] = pd.cut(df_completed['Hour'], bins=bins, labels=labels, right=False)
time_of_day = df_completed.groupby('Time of Day').agg({'Booking ID': 'count'}).reset_index()
time_of_day['% of Total'] = time_of_day['Booking ID']/time_of_day['Booking ID'].sum()*100
time_of_day['% of Total'] = time_of_day['% of Total'].round(2)
time_of_day.rename(columns={'Booking ID': 'Total Rides'}, inplace=True)
display(time_of_day.sort_values('Total Rides', ascending=False))

/var/folders/_6/k5gdhv996c7d1c62dptcsljm0000gn/T/ipykernel_84699/1409731981.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completed['Hour'] = df_completed['Date'].dt.hour
/var/folders/_6/k5gdhv996c7d1c62dptcsljm0000gn/T/ipykernel_84699/1409731981.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completed['Time of Day'] = pd.cut(df_completed['Hour'], bins=bins, labels=labels, right=False)
/var/folders/_6/k5gdhv996c7d1c62dptcsljm0000gn/T/ipykernel_84699/1409731981.py:5: FutureWarning: The defau

,Time of Day,Total Rides,% of Total
3,Evening,30602,32.91
2,Afternoon,29911,32.16
1,Morning,26543,28.54
0,Night,5944,6.39
